
# Nonlinear Equations


## Outline

1. Setup
2. Bisection method
3. Function iteration
4. Newton's method
5. Quasi-Newton method
6. Choosing a solution method
7. Convergence
8. Resources

## Setup

The nonlinear equation takes one of two forms.

* **Rootfinding problem** Given a function $f: R^n \mapsto R^n$,compute an $n$-vector $x^*$, called root of $f$, such that $f(x^*) = 0$.

* **Fixed-point problem** Given a function $g: R^n \mapsto R^n$, compute an $n$-vector $x^*$ called fixed point of $g$ such that $g(x^*) = x^*$

The two forms are equivalent.

* A root of $f$ is a fixed-point of $g(x) = x - f(x)$.
* A fixed-point of $g$ is a root ot $f(x) = x - g(x)$.

Nonlinear equations arise naturally in economics:

* Multicommodity market equilibrium models
* Multiperson static game models 
* Unconstrained optimization models

Nonlinear equations also arise indirectly when numerically solving economic models involving functional equations:

* Dynamic optimization models 
* Rational expectations models 
* Arbitrage pricing models

Building on our earlier lecture on the solution to linear systems of equations, often the solution to a nonlinear problem is computed iteratively by solving a sequence of linear problems. 

* sensitivity to intial condition
* susceptible to rounding error

In [1]:
#from temfpy.nonlinear_equations import exponential  # noqa: F401
from scipy import optimize
import pandas as pd
import numpy as np

from nonlinear_algorithms import bisect
from nonlinear_plots import plot_bisection_test_function

from nonlinear_algorithms import fixpoint
from nonlinear_plots import plot_fixpoint_example
from nonlinear_plots import plot_newton_pathological_example
from nonlinear_plots import plot_convergence
from nonlinear_plots import plot_newtons_method
from nonlinear_plots import plot_secant_method

from nonlinear_algorithms import newton_method

from nonlinear_problems import bisection_test_function
from nonlinear_problems import function_iteration_test_function
from nonlinear_problems import newton_pathological_example

## Bisection method

We start with the implementation (and testing) of the bisection algorithm.


* The bisection method is perhaps the simplest and most robust method for computing the root of a continuous real-valued function defined on a bounded interval of the real line. The bisection method is based on the Intermediate Value Theorem, which asserts that if a continuous real-valued function defined on an interval assumes two distinct values, then it must assume all values in between. In particular, if $f$ is continuous, and $f (a)$ and $f (b)$ have different signs, then $f$ must have at least one root $x$ in $[a, b]$.

* Each iteration begins with an interval known to contain or to bracket a root of $f$, because the function has different signs at the interval endpoints. The interval is bisected into two subintervals of equal length. One of the two subintervals must have endpoints of different signs and thus must contain a root of $f$. This subinterval is taken as the new interval with which to begin the subsequent iteration. In this manner, a sequence of intervals is generated, each half the width of the preceding one, and each known to contain a root of $f$. The process continues until the width of the bracketing interval containing a root shrinks below an acceptable convergence tolerance.

Let's visualize a test function to get a sense of what result can should expect.

* The ability to vectorize functions using [np.vectorize](https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html) comes in very handy when evaluating functions over a grid. However, be aware that this is a simple replacement of a `for loop` and thus does not improve performance. 

We are ready to go ...

Now we are ready to check our implementation and investigate the sensitivity of results to alternative tuning parameter.

### _Exercises_ 

1. Write a short test that ensures that in fact found a root of the function. 
2. Create a simple plot that shows each iterate of $x$ and label the two axis appropriately. 


## Function iteration


* Function iteration is a relatively simple technique that may be used to compute a fixed point $x = g(x)$ of a function from $R^n$ to $R^n$. The technique is also applicable to a rootfinding problem $f(x) = 0$ by recasting it as the equivalent fixed-point problem $g(x) = x - f (x)$. Function iteration begins with the analyst supplying a guess $x^{(0)}$ for the fixed point of $g$. Subsequent iterates are generated using the simple iteration rule

$$
\begin{align*}
x^{(k+1)} \leftarrow g(x^{(k)})
\end{align*}
$$

Since $g$ is continuous, if the iterates converge, they converge to a fixed point of $g$.

Let's visualize a test function to get a sense of what result can should expect.


Let's run and test our implementation.

* This is an example of using a try-except block to explicitly handle the `AssertionError`. Details on exception handling in Python, see this [Wiki](https://wiki.python.org/moin/HandlingExceptions) for some more examples. 

We set two parameters manually, which one is it? How about choosing different starting values?

Let's look at the `tolerance` setting instead.

### _Exercises_

1. Find the fixpoint of $g(x) = \sqrt{x + 0.2}$. 
2. How many iterations does the function iteration need? Does it depend on the starting value? 


## Newton's method


Newton's method is an algorithm for computing the root of a function $f : R^n \mapsto R^n$. Newton's method employs a strategy of successive linearization. The strategy calls for the nonlinear function $f$ to be approximated by a sequence of linear functions whose roots are easily computed and, ideally, converge to the root of $f$. In particular, the $k + 1^{th}$ iterate 

$$
\begin{align*}
x_{k+1} = x_k - f^\prime(x_k)^{-1}f(x_k) 
\end{align*}
$$

is the root of the Taylor linear approximation of $f$ around the preceding iterate $x_k$.

$$
\begin{align*}
f(x) \approx f(x_k) + f^\prime(x_k)(x - x_k).
\end{align*}
$$

This yields the following iteration rule:

$$
\begin{align*}
x_{k + 1} = x_k - f^\prime(x_k)^{-1} f(x_k)
\end{align*}
$$

If $n = 1$, the iteration rule takes the simpler form 

$$
\begin{align*}
x_{k + 1} = x_k - \frac{f(x_k)}{f^\prime(x_k)}.
\end{align*}
$$

### _Exercises_

1. Plot the function $f(x) = x^4 - 2$ and provide a rough estimate of its fix point. 
2. Let's compute the root of the function using Newton's method by employing the proper iteration rule: 
$$\begin{align*}
x_{k + 1} = x_k - \frac{f(x_k)}{f^\prime(c_k)} = x_k - \frac{x^4_k - 2}{4x^3_k}
\end{align*}$$
      




We now look at a more general implementation.

### _Question_

* What are the important generalizations?

Let' explore one last test function to test the interface of our function and learn something about [private functions](https://dbader.org/blog/meaning-of-underscores-in-python) in the process.

$$
\begin{align*}
f(x) = x^3 - 2
\end{align*}
$$

A potential shortcoming of Newton's method  is that the derivatives required for the Jacobian may not be available may be difficult to calculate analytically, or time-consuming to approximate numerically ... or that it might actually fail or result in cycles.

Consider the following pathological example which has a unique root at $0$.

$$
\begin{align*}
f(x) = x^{\frac{1}{3}} \, e^{-x^2}
\end{align*}
$$

We can derive the Newton iterates as:

$$
\begin{align*}
x_{k + 1} = x_k\left(1 - \frac{3}{1 - 6 x_k^2 }\right)
\end{align*}
$$

### _Questions_

1. What happens when you apply Newton's method to solve this function for different starting values?
2. What does the Newton iterate look like for very small and very large values?


* For $x_k$ small, the iteration reduces to $x_{k + 1} = -2x_k$ and so it converges to $0$ only if $x_0 = 0$.

* For $x_k$ large, the iteration becomes 

\begin{align*}
x_{k + 1} = x_k \left(1 + \frac{3}{x_k^2}\right)
\end{align*}

which diverges, but will eventually satisfy the stopping rule.

Another problems might be cycles.

<img src="material/fig-newton-cycles.png" width=600 height=600 />


## Quasi-Newton method


Quasi-Newton methods replace the Jacobian in Newton’s method with an estimate that is easier to compute. Specifically, quasi-Newton methods use an iteration rule

$$
\begin{align*}
x_{k+1} = x_k - A^{-1}_k f(x_k)
\end{align*}
$$

where $A_k$ is an estimate of the Jacobian $f^\prime(x_k)$. 

The **secant method** replaces the derivative in Newton’s method with the estimate

$$
\begin{align*}
f^\prime(x_k) \approx \frac{f(x_k) - f(x_{k-1})}{x_k - x_{k-1}}
\end{align*}
$$

The secant method is so called because it approximates the function $f$ using *secant* lines drawn through successive pairs of points on its graph.

Let's start with a crude implementation to make sure we understood the setup correctly. We will use this to show the `lambda` functions, see [here](https://realpython.com/python-lambda) for details.

**Broyden’s method** is the most popular multivariate generalization of the univariate secant method. **Broyden’s method** replaces the Jacobian in Newton’s method with an estimate $A_k$ that is updated by making the smallest possible change (measured by the Frobenius norm) that is consistent with the secant condition:

$$
\begin{align*}
f(x_{k + 1}) - f(x_k) = A_{k+1} (x_{k+1} - x_k ).
\end{align*}
$$

This yields the iteration rule

$$
\begin{align*}
A_{k+1} = A_k + (f(x_{k+1}) - f(x_k) - A_k d_k)\frac{d^\prime_k}{d^\prime_k d_k}
\end{align*}
$$

where $d_k = x_{k+1} - x_k$. Often $A_0$ is equal to the numerical approximation of $f$ at $x_0$. The remarkable feature of Broyden's method is that it is able to generate a reasonable approximation to the Jacobian matrix with no additional evaluations of the function. This approach is readily available in [scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.broyden1.html).




#### Cournot problem

Consider a market with two firms producing the same good. Firm $i$'s total cost of production is a function of the quantity $q_i$ it produces.

$$
\begin{align*}
C_i(q_i) = \frac{\beta_i}{2} q_i^2
\end{align*}
$$

The market clearing price is a function of the total quantity produced by both firms.

$$
\begin{align*}
P(q_1 + q_2) = (q_1 - q_2)^{-\alpha}
\end{align*}
$$

Firm $i$ chooses production $q_i$ so as to maximize its profit taking the other firm's output as given.

$$
\begin{align*}
\pi_i (q_1 , q_2 ) = P (q_1 + q_2 )q_i - C_i (q_i ).
\end{align*}
$$

Thus in equilibrium,

$$
\begin{align*}
\frac{\partial \pi_i}{\partial q_i} = (q_1 - q_2)^{-\alpha} - \alpha (q_1 + q_2)^{-(\alpha + 1) } q_i  - \beta_i q_i = 0\qquad\text{for}\quad i = 1, 2.
\end{align*}
$$

### _Exercise_

* Compute the market equilibrium quantities using Broyden's method for $\alpha =  0.6$ and $\beta= [0.6, 0.8]$.



<img src="material/fig-convergence-path.png" width=600 height=600 />


## Choosing a solution method



We now consider a more challenging task and compare the performance of `scipy`'s root finding algorithms. We will use one of [temfpy](https://temfpy.readthedocs.io/en/latest/index.html)'s test functions for this.

Now let's see which of the algorithms performs best for a ten dimensional problem.

We are ready to run our benchmarking exercise.

Now we can explore the performance of the alternative implementations.


## Convergence
        

Two factors determine the speed with which a properly coded and initiated algorithm will converge to a solution:

* Asymptotic rate of convergence
* Computational effort per iteration

The asymptotic rate of convergence measures improvement afforded per iteration near the solution. A sequence $x_k$ converges to $x^*$ at an asymptotic rate of order $p$ if there is constant $C > 0$ such that for $k$ sufficiently large,

$$
\begin{align*}|
||x_{k+1} - x^* || \leq C ||x_k - x^* ||^p.
\end{align*}
$$

* Function iteration converges at a “linear” rate with $p = 1$ and $C < 1$. 
* Broyden’s method converges at a “superlinear” rate with $p \approx 1.62$.
* Newton’s method converges at a “quadratic” rate with $p = 2$.

However, algorithms differ in computations per iteration.

* Function iteration requires a function evaluation.
* Broyden’s method additionally requires a linear solve.
* Newton’s method additionally requires a Jacobian evaluation.

Thus, a faster rate of convergence typically can be achieved only by investing greater computational effort per iteration. The optimal tradeoff between rate of convergence and computational effort per iteration varies across applications.

## Resources

#### Software

* **HOMPACK**: https://www.netlib.org/hompack